# Common Boggle Words

Let's find the most common words that appear in Boggle boards. There are about $16! \cdot 6^{16} \approx 5.9 \times 10^{25}$ different boards, not taking duplicate letters into account, such that we must recourse to sampling to in order to find word probabilities.

Boggle is a game with a square board, with 16 slots for cubes. Each cube has letters on its 6 faces. At the start of each game, the board is shuffled so that each cube is in a random slot, and each cube has a random face showing.

The goal of the game is to find as many chains of showing letters as possible on the resultant board. A player's score is based on the number of words that no other player found (larger unique words get higher scores). To be precise, a word chain must be a non-repeating sequence of horizontally, vertically, or diagonally contiguous cubes. Now, the same letter may appear twice in a path, but not the same cube. The words must belong to an agreed upon dictionary, and have some minimum length.

## Simulating Boggle

Here are the 16 cubes one will find in a standard Boggle board:

In [1]:
cubes = [('m', 'u', 'qu', 'h', 'n', 'i'),
         ('n', 'g', 'e', 'e', 'a', 'a'),
         ('d', 'r', 'y', 'v', 'l', 'e'),
         ('t', 't', 'r', 'e', 'y', 'l'),
         ('o', 'a', 'b', 'b', 'o', 'j'),
         ('h', 'w', 'v', 'e', 'r', 't'),
         ('s', 's', 'o', 'i', 't', 'e'),
         ('s', 'h', 'a', 'p', 'c', 'o'),
         ('g', 'e', 'w', 'n', 'e', 'h'),
         ('u', 'm', 'c', 'o', 't', 'i'),
         ('n', 'h', 'n', 'l', 'z', 'r'),
         ('t', 'w', 'a', 'o', 't', 'o'),
         ('p', 's', 'a', 'f', 'f', 'k'),
         ('t', 's', 't', 'i', 'd', 'y'),
         ('e', 'r', 'l', 'i', 'x', 'd'),
         ('s', 'u', 'e', 'e', 'n', 'i')]

To simulate the process of shuffling a board, both the slot and face of each cube will be chosen uniformly.

In [2]:
from random import choice, shuffle
from itertools import islice


def rand_board():
    board = []
    shuffle(cubes)
    cube_order = iter(cubes)
    for i in range(4):
        row = islice(cube_order, 4)
        board.append([choice(cube) for cube in row])
    return board

In [3]:
rand_board()

[['n', 'c', 'e', 't'],
 ['s', 'e', 'n', 's'],
 ['e', 'i', 's', 'd'],
 ['a', 'o', 'l', 't']]

To find viable words on a board, we'll recursively traverse adjoining letters from each of the 16 cube slots. Once a sequence goes out of bounds or repeats a cube, traversal in that direction will stop. 

Now, at each step in this process we could look up the sequence in a hash table (dictionary). But, it may be the case that the current sequence is no prefix of any dictionary word. So, it will be much more efficient to store the dictionary in a trie. [Here](http://stackoverflow.com/a/11016430) is an inspiring elegant implementation.

In [4]:
def make_trie(words):
    root = {}
    for word in words:
        node = root
        for letter in word:
            node = node.setdefault(letter, {})
        node[None] = None    # signify that node is a word
    return root

In [5]:
import json


t = make_trie(['apply', 'apple', 'apt'])
print(json.dumps(t, indent=2))

{
  "a": {
    "p": {
      "p": {
        "l": {
          "y": {
            "null": null
          },
          "e": {
            "null": null
          }
        }
      },
      "t": {
        "null": null
      }
    }
  }
}


Now let's write a recursive function that generates unique words on a given board, of at least some length.

In [6]:
from itertools import product


def get_words(board, trie, min_len=3):
    '''Yield possible words from a given board and trie.'''
    offsets = [(-1, -1), (-1, 0), (-1, 1),
               (0,  -1),          (0,  1),
               (1,  -1), (1,  0), (1,  1)]
    
    def recur(i, j, path, seen, node):
        if (i, j) in seen:    # path can't cross itself
            return
        if not (0 <= i < 4) or not (0 <= j < 4):    # path can't wrap around board
            return
        letter = board[i][j]
        path += letter
        seen.add((i, j))
        if letter in node:    # path is a prefix
            if None in node[letter]:    # path is a word
                yield path
            for i_off, j_off in offsets:    # traverse neighbors even if path is word
                yield from recur(i + i_off, j + j_off, path, seen.copy(), node[letter])
    
    seen = set()
    for i, j in product(range(4), range(4)):
        for word in recur(i, j, '', set(), trie):
            if len(word) >= min_len and word not in seen:
                yield word
                seen.add(word)

Finding words necessitates a dictionary, and a _Scrabble_-esque dictionary seems fitting. Conveniently, the analagous _Words With Friends_ game's dictionary is [publicly available](http://gaming.stackexchange.com/a/7163): the Enhanced North American Benchmark Lexicon. So I'm going to use that.

In [7]:
with open('enable1.txt') as f:
    trie = make_trie(word.strip() for word in f)

## Estimating Word Probabilities

To estimate word frequencies, we'll gather relative frequencies from random boards. But how many boards to sample from?

We can view the appearance of a specific word as a Bernoulli random variable with some unknown probability. We'd like to sample enough boards so that the resultant $k$ most common words is are in fact the most common, and in order.

In the land of probability bounds, where Markov and Chebyshev wander wide, one called Hoeffding's inequality is particularly powerful when it comes to Bernoulli variables. It can tell us how tight an $\epsilon$-sized confidence interval should be with $n$ samples.

Specifically, let $X_1, \ldots, X_n \sim \mathrm{Bernoulli}(p)$. Then, for any $\epsilon > 0$,

$$P(\left| \overline{X_n} - p \right| > \epsilon) \le 2 e^{-2n\epsilon^2}$$

So, to be $\alpha$ confident that some variable is within error $\epsilon$ of it's true value, we'll need

$$n \ge \frac{1}{2\epsilon^2}\log{\frac{2}{\alpha}}$$

samples.

This applies to any Bernoullie variable, and each random Boggle board gives a Bernoulli value to every word (it appears or it doesn't). So, we can just sample a bunch of boards, which will probably produce a tight enough bound.

However, we'll have to be very diligent about choosing our confidence $\alpha$, and tightness $\epsilon$. We can't guarantee that the top $k$ words are in fact the top $k$, and in order, but it does ossify those top probabilities with a diligently chosen $\epsilon$.

Experimenting with 1000 boards showed that top 10 words' probabilities overlap, suggesting that we need $\epsilon < 0.001$ to be, say, $\alpha = 0.1$ confident that the ordering is correct. Unfortunately, this would lead to $n \ge 1.5 \times 10^6$ boards, which is too many hours of computation. Instead we'll shoot for just having the same set of top words, rather than correct ranking. The difference in probability between the empirical most common word and the 20th (we choose $k$ arbitrarily) was about 0.01, yielding $n \ge 14979$ boards, which is more manageable.

In [20]:
%%time
from collections import Counter
from tabulate import tabulate
import multiprocessing as mp

NBOARDS = 14979


def update_counts(_):
    board = rand_board()
    return Counter(get_words(board, trie, min_len=4))


counts = Counter()
nprocs = mp.cpu_count()
with mp.Pool(processes=nprocs) as pool:
    it = pool.imap_unordered(update_counts, range(NBOARDS),
                             chunksize=500)
    for sampled_counts in it:
        counts += sampled_counts


table = [(i, word, count/NBOARDS) for i, (word, count)
         in enumerate(counts.most_common(20), start=1)]
print(tabulate(table, headers=['Rank', 'Word', 'Frequency'], floatfmt='.4f'))

  Rank  Word      Frequency
------  ------  -----------
     1  tees         0.0648
     2  teen         0.0647
     3  tent         0.0602
     4  tone         0.0601
     5  note         0.0591
     6  teat         0.0575
     7  toea         0.0556
     8  nets         0.0549
     9  test         0.0545
    10  neat         0.0536
    11  toes         0.0533
    12  sent         0.0532
    13  nest         0.0531
    14  tens         0.0530
    15  rete         0.0521
    16  thee         0.0515
    17  tets         0.0512
    18  stet         0.0512
    19  nett         0.0512
    20  eats         0.0503
CPU times: user 47.5 s, sys: 300 ms, total: 47.8 s
Wall time: 1min 2s


## Common and Ordinary Word Ranking

The most common list is interesting, but some of the words are so ordinary that other players will probably know them and also find them. To improve one's chances of winning Boggle, it would be nice to have a ranking that takes into account both a word's probability of occurence, *and ordinariness*.

This is reminiscent [tf-idf weighting](http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html)! Our current list gives us the *term frequency*, so all that remains is to find each term's *inverse document frequency*. Here, we assume that usage corresponds to how likely it is for a word to be known.

We'll use word usage counts from [American National Corpus](http://www.anc.org/data/anc-second-release/frequency-data/). However, some of the words from Boggle might have various endings (e.g. "apple", "apples"), and those should add in frequency for "ordinariness." So we'll stem the words for lookup.

In [10]:
from nltk.stem import SnowballStemmer
from collections import defaultdict


snow = SnowballStemmer('english')
freqs = defaultdict(int)
min_freq = 1    # smallest frequency found, for use with Boggle words not in list

with open('ANC-token-count.txt', encoding='ISO-8859-1') as f:    # there's weird characters
    for word, *_, freq in map(str.split, f):
        if word == 'Total':    # there was a total count of words at the end
            continue
        word, freq = snow.stem(word), float(freq)
        freqs[word] += freq
        min_freq = freqs[word]

Now, we just need to calculate the scores of the words output a new sorted list.

In [11]:
from math import log


tfs, idfs = [], []
for word, count in counts.items():
    word = snow.stem(word)
    tfs.append(1 + log(count))
    idfs.append(log(1 + 1/freqs.get(word, min_freq)))

scores = sorted([(tf*idf, word) for tf, idf, word in zip(tfs, idfs, counts.keys())], reverse=True)
table = [(i, word, score) for i, (score, word) in enumerate(scores[:20], start=1)]
print(tabulate(table, headers=['Rank', 'Word', 'Score']))

  Rank  Word      Score
------  ------  -------
     1  seel    55.86
     2  stet    54.0779
     3  hest    52.0857
     4  sett    49.8272
     5  yett    47.2199
     6  tews    47.2199
     7  roti    47.2199
     8  heth    47.2199
     9  hent    47.2199
    10  haet    47.2199
    11  thio    45.2848
    12  teth    45.2848
    13  sone    45.2848
    14  erst    45.2848
    15  teel    44.1361
    16  seta    44.1361
    17  sego    44.1361
    18  osar    44.1361
    19  naoi    44.1361
    20  hoer    44.1361


# Epilogue

After completing this task, several sub-topics came to mind. We address some of those here. Unlike in previous sections, we will be lax proving the correct number of boards.

## Number of Possible Words Distribution

It would be interesting to see the minimum, maximum, mean, etc. of the number of words found over all boards. For the sake of brevity, rather than fitting the distribution and devising estimators, we'll just plot a simple histogram. And I choose an arbitrarily large number of boards to sample from.

In [ ]:
import matplotlib.pyplot as plt
import seaborn


%matplotlib inline
seaborn.set_context('notebook')
seaborn.set_style('whitegrid')

num_words = []
for _ in range(2000):
    num_words.append(len(list(get_words(rand_board(), trie))))
    
plt.hist(num_words, bins=40, normed=True)
plt.title('Number of Possible Words Distribution')
plt.xlabel('Number of words')
plt.ylabel('Proportion')
plt.show()

## Best Proportion of Vowels

A friend gave the comment that the "best" Boggle boards are comprised of 40% vowels. Best, here, was intended to mean highest number of word solutions. Now to experimentally prove this claim.

In [ ]:
from itertools import chain
from collections import defaultdict


NBOARDS = 1000
data = defaultdict(int)    # store how many words found per vowel, summed over boards
vowels = set('aeiou')    # for simplicity we assume that these are the only vowels

for _ in range(NBOARDS):
    num_vowels = 0
    board = rand_board()
    for letter in chain(*board):
        if letter in vowels:
            num_vowels += 1
    num_words = len(list(get_words(board, trie, min_len=4)))
    data[num_vowels] += num_words

Now to plot the mean number of found words per number of vowels:

In [ ]:
x, y = zip(*data.items())

plt.plot(x, [t/NBOARDS for t in y], 'bo:')
plt.title(r'Number of Words in Board per Vowel')
plt.xlabel('Number of vowels')
plt.ylabel('Total number of found words')
plt.show()

In [ ]:
6/16

Would you look at that, 40% was pretty spot on for an off the cuff comment.

## Super Big Boggle

In [54]:
big_cubes = [
    ('s', 's', 's', 'u', 'n', 'e'),
    ('qu', 'j', 'k', 'z', 'w', 'x'),
    ('a', 'a', 'a', 's', 'f', 'r'),
    ('h', 'o', 'r', 'd', 'l', 'n'),
    ('g', 'o', 'r', 'w', 'r', 'v'),
    ('f', 's', 'i', 'r', 'a', 'a'),
    ('an', 'in', 'er', 'he', 'th', 'qu'),
    ('a', 'e', 'a', 'e', 'e', 'e'),
    ('l', 'i', 'e', 'a', 'n', 'm'),
    ('g', 'f', 'n', 'u', 'y', 'c'),
    ('a', 'e', 'm', 'e', 'e', 'e'),
    ('h', 'd', 't', 'n', 'o', 'd'),
    ('h', 's', 'e', 'r', 'i', 'l'),
    ('n', 'u', 'e', 'o', 'i', 'a'),
    ('o', 'n', 'd', 'w', 'h', 'h'),
    ('a', 'o', 'i', 'e', 'b', 'd'),
    ('l', 'h', 'h', 'r', 'd', 'o'),
    ('g', 'n', 'a', 'm', 'n', 'e'),
    ('s', 't', 'r', 'h', 'p', 'o'),
    ('n', 'd', 'e', 'a', 'n', 'n'),
    ('o', 'o', 'a', 'e', 'a', 'e'),
    ('t', 'm', 't', 'e', 'o', 't'),
    ('t', 'i', 's', 'r', 'v', 'h'),
    ('a', 'e', 'm', 'g', 'u', 'e'),
    ('t', 'i', 'l', 's', 'p', 'e'),
    ('n', 'o', 'w', 'o', 'u', 't'),
    ('y', 'r', 'y', 's', 'i', 'p'),
    ('e', 's', 'p', 't', 'i', 'c'),
    ('c', 't', 'n', 's', 'e', 'c'),
    ('y', 'r', 'f', 's', 'i', 'a'),
    ('l', 'c', 'd', 'd', 'n', 'n'),
    ('t', 'i', 't', 'c', 't', 'e', 'i'),
    ('e', 't', 'i', 'l', 'i', 's'),
    ('j', 'b', 'b', 'z', 'x', 'k'),
    ('o', 'o', 'o', 't', 'u', 't'),
    ('o', 'i', None, None, 'e', None)
]